In [6]:
import torch
from torch_geometric.datasets import TUDataset
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch.utils.data import random_split
from experiments.classifier import GraphClassifier
from experiments.train import train, test
from torch.optim import Adam

# Load the dataset
dataset_name = "ENZYMES"
path = "./data/tmp/ENZYMES"
original_dataset = TUDataset(root=path, name=dataset_name, use_node_attr=True)

# Function to add equal edge weights to a graph
def add_edge_weights(data):
    edge_index = data.edge_index
    edge_weight = torch.ones(edge_index.shape[1], dtype=torch.float)
    return Data(edge_index=edge_index, x=data.x, y=data.y, edge_weight=edge_weight)

# Create a new dataset with added edge weights
dataset = [add_edge_weights(data) for data in original_dataset]

# Split the dataset into train and test sets
train_size = int(0.8 * len(dataset))  # 80% of the dataset for training
test_size = len(dataset) - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create DataLoaders for train and test sets
train_loader = DataLoader(train_dataset, batch_size=2048, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=2048, shuffle=False)

n_input = dataset[0].x.size(1)

# Set the device to use for training (either 'cpu' or 'cuda' if GPU is available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [7]:
ts = torch.linspace(0, 1, 20, dtype=torch.float, device=device)
# ts = torch.tensor([0.5], dtype=torch.float, device=device)
# Create your model, optimizer, and criterion
# model = GraphClassifier(pde='heat', ts=ts, n_input=n_input, n_hidden=16, n_output=6, n_time_conv=5, n_time_pool=5, device=device).to(device)
model = GraphClassifier(pde='heat', ts=ts, n_input=n_input, n_hidden=16, n_output=6, device=device).to(device)
optimizer = Adam(model.parameters(), lr=1e-3)
# criterion = CrossEntropyLoss()
criterion = torch.nn.NLLLoss()

# Train the model
train(model, train_loader, optimizer, criterion, device, num_epochs=1000)
test(model, test_loader, device)

Epoch 1/1000, Loss: 1.8461, Accuracy: 0.1667
Epoch 101/1000, Loss: 1.4782, Accuracy: 0.4042


In [ ]:
# ts = torch.linspace(torch.pi/2, torch.pi, 20, dtype=torch.float, device=device)
ts = torch.tensor([torch.pi/2], dtype=torch.float, device=device)
# Create your model, optimizer, and criterion
# model = GraphClassifier(pde='wave', ts=ts, n_input=n_input, n_hidden=16, n_output=6, n_time_conv=5, n_time_pool=5, device=device).to(device)
model = GraphClassifier(pde='wave', ts=ts, n_input=n_input, n_hidden=16, n_output=6, device=device).to(device)
optimizer = Adam(model.parameters(), lr=1e-3)
# criterion = CrossEntropyLoss()
criterion = torch.nn.NLLLoss()

# Train the model
train(model, train_loader, optimizer, criterion, device, num_epochs=1000)
test(model, test_loader, device)

Epoch 1/1000, Loss: 4.4857, Accuracy: 0.1333
Epoch 101/1000, Loss: 1.7088, Accuracy: 0.2458
Epoch 201/1000, Loss: 1.6597, Accuracy: 0.2917
Epoch 301/1000, Loss: 1.6003, Accuracy: 0.3563
Epoch 401/1000, Loss: 1.5220, Accuracy: 0.3875
Epoch 501/1000, Loss: 1.4546, Accuracy: 0.4062
Epoch 601/1000, Loss: 1.3712, Accuracy: 0.4417
Epoch 701/1000, Loss: 1.3104, Accuracy: 0.4750
Epoch 801/1000, Loss: 1.2253, Accuracy: 0.5229
Epoch 901/1000, Loss: 1.2037, Accuracy: 0.5188
Test accuracy: 46.67%


In [ ]:
from experiments.gcn import GCNClassifier
# Create your model, optimizer, and criterion
model = GCNClassifier(num_features=n_input, hidden_channels=16, num_classes=6)
optimizer = Adam(model.parameters(), lr=1e-2)
# criterion = CrossEntropyLoss()
criterion = torch.nn.NLLLoss()

# Train the model
train(model, train_loader, optimizer, criterion, device, num_epochs=1000)
test(model, test_loader, device)

Epoch 1/1000, Loss: 5.0149, Accuracy: 0.1604
Epoch 101/1000, Loss: 1.6880, Accuracy: 0.2687
Epoch 201/1000, Loss: 1.6360, Accuracy: 0.3146
Epoch 301/1000, Loss: 1.5634, Accuracy: 0.3563
Epoch 401/1000, Loss: 1.4615, Accuracy: 0.3979
Epoch 501/1000, Loss: 1.3975, Accuracy: 0.4458
Epoch 601/1000, Loss: 1.3205, Accuracy: 0.4792
Epoch 701/1000, Loss: 1.2762, Accuracy: 0.5104
Epoch 801/1000, Loss: 1.1821, Accuracy: 0.5542
Epoch 901/1000, Loss: 1.1445, Accuracy: 0.5708
Test accuracy: 42.50%
